In [7]:
import pandas as pd 
import psycopg2
import plotly.express as px
import datetime
# import os


filter_broker = ['Necton','Guide','Bradesco']
today = datetime.date.today()

# display(df_saldo)


def get_analysis(ticker):
    db_conn = psycopg2.connect(host='PGKPTL01', dbname='db_Teste' , user='kapitalo11', password='kapitalo11')
    query = f"select * from ibotz.tbl_boletasalugueis_ibotz where str_codigo in {ticker} and str_corretora not like '%Interna%' and str_mercado not like '%Devolucao%' and dbl_quantidade<0"
    df = pd.read_sql(query,db_conn)
    df['str_corretora'] = df['str_corretora'].apply(lambda x: x.replace('FC Stone','Necton'))
    db_conn.close()
    return df



In [16]:
brokers = ['XP']
restricted = ['BOVA11','BRFS3','GOLL4','GFSA3','CMIN3','LEVE3','AMPB3','TRPL4','CVCB3','ASAI3','ROXO34']
def build_loan_files(filter_broker=brokers,restricted_tickers =restricted):

    df_saldo = pd.read_excel(f"G:\Trading\K11\Aluguel\Arquivos\Doar\Saldo-Dia\K11_lend_complete_{datetime.date.today().strftime('%d-%m-%Y')}.xlsx",index_col=0)
    df_saldo['cod'] = df_saldo['codigo'].apply(lambda x: x[:4])
    df = get_analysis(tuple(df_saldo['cod'].unique()))


    df = df.groupby(['str_corretora','str_codigo']).agg({'dbl_quantidade':sum}).reset_index().set_index('str_codigo').sort_values('dbl_quantidade')


    for x in df_saldo['cod'].unique():
        try:
            df.loc[x,'prop'] = df.loc[x,'dbl_quantidade']/df.loc[x,'dbl_quantidade'].sum()
            # fig_a= px.bar(df.loc[x].head(),y="prop",x="str_corretora",title=x)
            # fig_a.show()
            
        except:
            pass

    df_saldo = df_saldo.merge(df,left_on='cod',right_on='str_codigo',how='left')

    corretoras = pd.read_excel(r'G:\Transfer\Lista de Contrapartes Autorizadas\Contrapartes Operacionais.onshore.xlsx',sheet_name='BTC',skiprows=7)

    
    corretoras = pd.read_excel(r'G:\Transfer\Lista de Contrapartes Autorizadas\Contrapartes Operacionais.onshore.xlsx',sheet_name='BTC',skiprows=7)
    corretoras = corretoras[['Contraparte', 'Master II ', 'Master I ',
        'Tarkus Master ', 'Argus Master', 'Sigma LLC ', 'Gaia',
        'Alpha Global Master', 'Estrategias Master', 'Master V',
        'Zeta Merídia Master', 'K10 Previdência', 'K10 Master', 'Ômega Prev.',
        'Kappa Prev. Master']].fillna(0)


    de_para = {
        'Contraparte':'Contraparte',
        'Master I ':'KAPITALO KAPPA MASTER FIM',
        'Master II ':'KAPITALO ZETA MASTER FIM',
        'Sigma LLC ':'KAPITALO SIGMA LLC',
        'Zeta Merídia Master':'KAPITALO ZETA MASTER FIA',
        'K10 Previdência' :'KAPITALO K10 PREV MASTER FIM',
        'K10 Master':'KAPITALO K10 MASTER FIM',
        'Ômega Prev.':'KAPITALO OMEGA PREV MASTER FIM',
        'Kappa Prev. Master':'KAPITALO KAPPA PREV MASTER FIM'
    }

    de_para_corretoras ={
        'Ativa Investimentos':'Ativa',
        'BGC Liquidez':'Liquidez',
        'BTG Pactual CTVM':'BTG Pactual',
        'Capital Markets':'Capital Markets',
        'Citigroup CCTVM':'Citi',
        'Credit Suisse':'CS',
        'Genial Institucional ':'Plural',
        'Guide Investimentos':'Guide',
        'Inter DTVM':'Inter',
        'INTL FCStone DTVM':'FC Stone',
        'Itaú Corretora':'Itau',
        'JP Morgan CCVM':'JP Morgan',
        'Merrill Lynch CTVM':'Bank of America',
        'Mirae Asset Wealth Management':'Mirae',
        'Morgan Stanley CTVM':'Morgan Stanley',
        'Necton Investimentos':'Necton',
        'Órama DTVM':'Orama',
        'Safra Corretora de Valores e Câmbio':'Safra',
        'Santander CCVM':'Santander',
        'Terra Investimentos DTVM':'Terra',
        'Tullett Prebon Brasil':'Tullet',
        'UBS BrasiL CCTVM':'Link',
        'XP Investimentos':'XP',
    }

    corretoras.columns = corretoras.columns.map(de_para)

    corretoras = corretoras[corretoras.columns.dropna()]
    corretoras = corretoras.set_index('Contraparte')

    corretoras = corretoras.reset_index()




    # corretoras

    corretoras['Contraparte'] = corretoras['Contraparte'].map(de_para_corretoras).dropna()
    corretoras = corretoras.dropna()


    corretoras = corretoras.reset_index()

    corretoras = corretoras[corretoras['Contraparte'].isin(filter_broker)]

    corretoras = corretoras.set_index('Contraparte')
    if 'Bradesco' in filter_broker:
        corretoras.loc['Bradesco'] = 'x'

    new_df = pd.DataFrame()
    for x in df_saldo['fundo'].unique():
        
        try:
            lista_corretoras = corretoras[corretoras[x]!=0][[x]].index
            aux = df_saldo.loc[(df_saldo['fundo']==x)&(df_saldo['str_corretora'].isin(lista_corretoras))].copy()
            new_df = pd.concat([new_df,aux])
        except:
            pass

    for fund in new_df['fundo'].unique():
        for x in new_df['codigo'].unique():
            new_df.loc[(new_df['fundo']==fund) & (new_df['codigo']==x),'prop'] = new_df.loc[(new_df['fundo']==fund) & (new_df['codigo']==x),'prop']/new_df.loc[(new_df['fundo']==fund) & (new_df['codigo']==x),'prop'].sum()
    new_df = new_df[new_df['prop']>0.30]
    for fund in new_df['fundo'].unique():
        for x in new_df['codigo'].unique():
            new_df.loc[(new_df['fundo']==fund) & (new_df['codigo']==x),'prop'] = new_df.loc[(new_df['fundo']==fund) & (new_df['codigo']==x),'prop']/new_df.loc[(new_df['fundo']==fund) & (new_df['codigo']==x),'prop'].sum()

    new_df['Doador'] = new_df.apply(lambda row: round(row['to_lend']*row['prop'],0),axis=1)



    geral = new_df[~new_df['codigo'].isin(restricted_tickers)].copy()
    restricted = new_df[new_df['codigo'].isin(restricted_tickers)].copy()

    # new_df['str_corretora'] = new_df['str_corretora'].apply(lambda x: x.replace('Link','Guide'))
    for x in new_df['str_corretora'].unique():
        output_file_path = f"G:\Trading\K11\Aluguel\Arquivos\Doar\\Saldo-Dia-brokers\\{today.strftime('%d-%m-%Y')}\\Kapitalo_K11_{x}_{datetime.date.today().strftime('%Y-%m-%d')}.xlsx"
        aux_x = geral[geral['str_corretora']==x][['fundo','codigo','Doador']].copy()
        aux_x = aux_x[aux_x['Doador']>100]
        if os.path.exists(
            f"G:\Trading\K11\Aluguel\Arquivos\Doar\\Saldo-Dia-brokers\\{today.strftime('%d-%m-%Y')}"
        ):

            aux_x.to_excel(output_file_path)
        else:
            os.mkdir(f"G:\Trading\K11\Aluguel\Arquivos\Doar\\Saldo-Dia-brokers\\{today.strftime('%d-%m-%Y')}")
            aux_x.to_excel(output_file_path)
    
        # restricted[restricted['str_corretora']==x][['fundo','codigo','Doador']].to_excel(f"G:\Trading\K11\Aluguel\Arquivos\Doar\\Saldo-Dia-brokers\\Kapitalo_K11__restricted{x}_{datetime.date.today().strftime('%Y-%m-%d')}.xlsx")
    output_file_path_r =  f"G:\Trading\K11\Aluguel\Arquivos\Doar\\Reservados\\{today.strftime('%d-%m-%Y')}\\Reservados_{datetime.date.today().strftime('%Y-%m-%d')}.xlsx"
    if os.path.exists(
            f"G:\Trading\K11\Aluguel\Arquivos\Doar\\Reservados\\{today.strftime('%d-%m-%Y')}"
        ):

        restricted.groupby(['fundo','codigo']).sum()[['to_lend']].to_excel(output_file_path_r)
    else:
        os.mkdir(f"G:\Trading\K11\Aluguel\Arquivos\Doar\\Reservados\\{today.strftime('%d-%m-%Y')}")
        restricted.groupby(['fundo','codigo']).sum()[['to_lend']].to_excel(output_file_path_r)


In [17]:
build_loan_files()

In [10]:
df_saldo = pd.read_excel(f"G:\Trading\K11\Aluguel\Arquivos\Doar\Saldo-Dia\K11_lend_complete_{datetime.date.today().strftime('%d-%m-%Y')}.xlsx",index_col=0)

In [3]:
display()
display(restricted.groupby(['codigo']).sum())

AttributeError: 'list' object has no attribute 'groupby'

In [ ]:
bova = pd.read_excel(r'G:\Trading\K11\CashCarry\bova11\bova11_2023-11-06.xls',skiprows=1)[['Unnamed: 0','Unnamed: 1']]
bova.columns = ['Nome','Data']
bova.loc[bova['Nome']=='Data','Data'].item()
bova.loc[bova['Nome']=='Valor líquido dos ativos por quota**','Data'].item()

In [157]:
import pyodbc
import sys
sys.path.append('../..')
import workdays
holidays_br = workdays.load_holidays('BR')
holidays_b3 = workdays.load_holidays('B3')


dt_1 = workdays.workday(datetime.date.today(), -1, holidays_br)
dt_2 = workdays.workday(datetime.date.today(), -2, holidays_br)
CORPORATE_DSN_CONNECTION_STRING = "DSN=Kapitalo_Corp"
aux = '"":""'
connection = pyodbc.connect(CORPORATE_DSN_CONNECTION_STRING)
df = pd.read_sql(f" CALL up2data.XSP_UP2DATA_DEFAULT('{dt_1.strftime('%Y-%m-%d')}', 'Equities_AssetLoanFileV2', '{aux}' )",connection)
df.columns = [x.lower() for x in df.columns]
df =df[df['mktnm']=='Balcao']

df_2 = pd.read_sql(f" CALL up2data.XSP_UP2DATA_DEFAULT('{dt_2.strftime('%Y-%m-%d')}', 'Equities_AssetLoanFileV2', '{aux}' )",connection)
df_2.columns = [x.lower() for x in df_2.columns]
df_2 =df_2[df_2['mktnm']=='Balcao']


df = df[['tckrsymb','takravrgrate','takrminrate','takrmaxrate','qtyshrday']]
df_2 = df_2[['tckrsymb','takravrgrate','takrminrate','takrmaxrate','qtyshrday']]


df.columns = ['codigo','taxa media','taxa minima','taxa max','negocios']

df_2.columns = ['codigo','taxa media d-2','taxa minima d-2','taxa max d-2','negocios d-2']


df['taxa media'] = df['taxa media'].astype(float)

df['taxa minima'] = df['taxa minima'].astype(float)

df['taxa max'] = df['taxa max'].astype(float)

df['negocios'] = df['negocios'].astype(float)

df_2['taxa media d-2'] = df_2['taxa media d-2'].astype(float)

df_2['taxa minima d-2'] = df_2['taxa minima d-2'].astype(float)

df_2['taxa max d-2'] = df_2['taxa max d-2'].astype(float)

df_2['negocios d-2'] = df_2['negocios d-2'].astype(float)


df = df.merge(df_2,on='codigo',how='inner')

df['dif taxa'] = (((df['taxa media'] / df['taxa media d-2']) -1)*100)

df['dif volume'] = abs((((df['negocios'] / df['negocios d-2']) -1)*100))

# Consider trades more expressive if they are in the top 25%
threshold = df['negocios'].quantile(0.9)

# Filter the DataFrame
filtered_df = df[df['negocios'] > threshold]


highter_taxes = filtered_df.sort_values('taxa media',ascending=False)
highter_taxes = highter_taxes[highter_taxes['dif taxa']>-2].head(7)['codigo'].tolist()

if 'BOVA11' not in highter_taxes:
    highter_taxes.append('BOVA11')
taxas = df[df['codigo'].isin(highter_taxes)][['codigo','taxa media','taxa max']]

In [169]:
df = pd.read_excel(r'G:\Trading\K11\Aluguel\Arquivos\Renovações\renovacao_20231108.xlsx')

In [ ]:
## Old - Devol

str_corretora, str_tipo, dte_datavencimento,dbl_taxa, str_reversivel, str_papel, dbl_quantidade, str_tipo_registro, str_modalidade, str_tipo_comissao, dbl_valor_fixo_comissao, str_status

In [176]:
df_devol = pd.DataFrame()

df_devol['int_codcontrato'] = renov['contrato']
df_devol['str_fundo'] = renov['str_fundo']

df_devol['dte_data'] = renov['data']

df_devol['dte_databoleta'] = datetime.date.today()

df_devol['str_corretora'] = renov['corretora']

df_devol['str_tipo'] = renov['tipo']

df_devol['dte_datavencimento'] = renov['vencimento']

df_devol['dbl_taxa'] = renov['taxa']

df_devol['str_reversivel'] = renov['reversivel']

df_devol['str_papel'] = renov['codigo']

df_devol['dbl_quantidade'] = renov['Quantidade']

df_devol['str_tipo_registro'] = ""

df_devol['str_modalidade'] = ""

df_devol['str_tipo_comissao'] = ""

df_devol['dbl_valor_fixo_comissao'] = 0

df_devol['str_status'] = "Devolucao-Renovacao"

df_renov = pd.DataFrame()


df_renov['str_fundo'] = renov['str_fundo']
df_renov['str_corretora'] = renov['corretora']

df_renov['dte_data'] = datetime.date.today()
df_renov['dte_databoleta'] = datetime.date.today()

df_renov['str_tipo'] = renov['tipo']

df_renov['dte_datavencimento'] = renov['Vencimento next']

df_renov['dbl_taxa'] = renov['taxa d-1']

df_renov['str_reversivel'] = renov['reversivel']

df_renov['str_papel'] = renov['codigo']

df_renov['dbl_quantidade'] = renov['Quantidade']

df_renov['str_tipo_registro'] = renov['tipo']

df_renov['str_modalidade'] = renov['registro']

df_renov['str_tipo_comissao'] = renov['comissao']

df_renov['dbl_valor_fixo_comissao'] = renov['fixo']


df_renov['str_status'] = "Emprestimo-Renovacao"

,Unnamed: 0,data,str_fundo,corretora,tipo,vencimento,taxa,preco,reversivel,codigo,contrato,saldo,Quantidade,taxa d-1,Vencimento next,Troca,registro,e1,comissao,fixo
0,0,2023-10-11,KAPITALO K10 PREV II MASTER FIM,Bradesco,T,2023-11-13,7.62,58.24,TD,M1TA34,2023101100516434690001-1,12,12,11.99,2023-12-19,NaN,R,NaN,A,0
1,1,2023-10-11,KAPITALO KAPPA PREV II MASTER FIM,Bradesco,T,2023-11-13,7.62,58.24,TD,M1TA34,2023101100516435590001-1,1,1,11.99,2023-12-19,NaN,R,NaN,A,0
2,2,2023-10-11,KAPITALO KAPPA PREV II MASTER FIM,Bradesco,T,2023-11-13,7.62,58.24,TD,M1TA34,2023101100516435140001-1,4,4,11.99,2023-12-19,NaN,R,NaN,A,0
3,3,2023-10-02,KAPITALO K10 PREV MASTER FIM,Bradesco,T,2023-11-13,9.11,54.33,TD,M1TA34,2023100200497704110001-2,608,608,11.99,2023-12-19,NaN,R,NaN,A,0
4,4,2023-10-02,KAPITALO K10 PREV MASTER FIM,Bradesco,T,2023-11-13,9.11,54.33,TD,M1TA34,2023100200497699690001-2,199,199,11.99,2023-12-19,NaN,R,NaN,A,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,339,2023-10-09,KAPITALO KAPPA PREV MASTER FIM,Bradesco,T,2023-11-13,2.99,6.27,TD,LWSA3,2023100900514684200001-1,109,109,3.83,2023-12-19,NaN,R,NaN,A,0
340,340,2023-10-11,KAPITALO KAPPA PREV MASTER FIM,Bradesco,T,2023-11-13,0.03,37.97,TD,PETR3,2023101100516435490001-1,13,13,0.15,2023-12-19,NaN,R,NaN,A,0
341,341,2023-10-02,KAPITALO KAPPA PREV MASTER FIM,Bradesco,T,2023-11-13,0.98,9.78,TD,VAMO3,2023100200497366780001-2,390,390,1.47,2023-12-19,NaN,R,NaN,A,0
342,342,2023-10-02,KAPITALO ZETA MASTER FIA,XP,D,2023-11-13,0.03,67.40,TD,VALE3,2023100200512387240001-1,-5000,-5000,0.58,2023-12-19,NaN,R,NaN,A,0


In [197]:
import config2 as config
import psycopg2
from io import StringIO
conn = psycopg2.connect(
host=config.DB_TESTE_HOST,
dbname=config.DB_TESTE_NAME,
user='rodrigo', password='cavenaghi'
)
cursor = conn.cursor()
sio = StringIO()
# Write the Pandas DataFrame as a csv to the buffer
sio.write(df_renov.to_csv(index=None, header=None, sep=";"))
sio.seek(0)  # Be sure to reset the position to the start of the stream
# Copy the string buffer to the database, as if it were an actual file
cursor.copy_from(sio, "tbl_novasboletasaluguel", columns=df_renov.columns, sep=";")
print(conn.commit())
conn.close()
conn = psycopg2.connect(
host=config.DB_TESTE_HOST,
dbname=config.DB_TESTE_NAME,
user='rodrigo', password='cavenaghi'
)
cursor = conn.cursor()
sio = StringIO()
# Write the Pandas DataFrame as a csv to the buffer
sio.write(df_devol.to_csv(index=None, header=None, sep=";"))
sio.seek(0)  # Be sure to reset the position to the start of the stream
# Copy the string buffer to the database, as if it were an actual file
cursor.copy_from(sio, "tbl_novasboletasaluguel", columns=df_devol.columns, sep=";")
print(conn.commit())
conn.close()

None


In [205]:
conn = psycopg2.connect(
host=config.DB_TESTE_HOST,
dbname=config.DB_TESTE_NAME,
user='rodrigo', password='cavenaghi'
)
cursor = conn.cursor()
sio = StringIO()
# Write the Pandas DataFrame as a csv to the buffer
sio.write(df_devol.to_csv(index=None, header=None, sep=";"))
sio.seek(0)  # Be sure to reset the position to the start of the stream
# Copy the string buffer to the database, as if it were an actual file
cursor.copy_from(sio, "tbl_novasboletasaluguel", columns=df_devol.columns, sep=";")
print(conn.commit())
conn.close()

None


In [163]:
filtered_df

,codigo,taxa media,taxa minima,taxa max,negocios,taxa media d-2,taxa minima d-2,taxa max d-2,negocios d-2,dif taxa,dif volume
4,ABEV3,0.11,0.01,1.14,4728625.0,0.14,0.10,1.14,7289224.0,-21.428571,35.128554
5,AERI3,0.40,0.01,1.39,6098726.0,1.39,1.39,1.39,732625.0,-71.223022,732.448524
10,ALOS3,0.11,0.01,0.12,1650389.0,0.12,0.10,0.14,2252845.0,-8.333333,26.742008
11,ALPA4,11.06,11.06,11.06,2200656.0,11.06,10.80,12.00,452011.0,0.000000,386.858948
16,AMBP3,33.43,0.01,33.50,2178342.0,33.43,0.01,33.44,130137.0,0.000000,1573.883676
25,ASAI3,9.06,0.01,15.00,4442560.0,5.41,0.01,15.00,10781117.0,67.467652,58.793138
34,B3SA3,0.05,0.01,1.09,10009357.0,0.09,0.01,0.20,17827751.0,-44.444444,43.855190
40,BBDC4,0.05,0.01,0.50,10304458.0,0.05,0.01,0.60,14747094.0,0.000000,30.125501
50,BHIA3,1.29,0.01,2.10,8685029.0,1.30,1.17,1.50,6521868.0,-0.769231,33.167813
61,BOVA11,0.56,0.01,3.00,2903669.0,0.57,0.01,1.50,3539939.0,-1.754386,17.974039


,codigo,taxa media,taxa minima,taxa max,negocios,taxa media d-2,taxa minima d-2,taxa max d-2,negocios d-2,dif taxa,dif volume
96,CMIN3,36.82,0.01,36.92,24220404.0,36.92,0.10,36.92,3258460.0,-0.270856,643.308311
16,AMBP3,33.43,0.01,33.50,2178342.0,33.43,0.01,33.44,130137.0,0.000000,1573.883676
72,BRFS3,32.52,0.01,40.00,8603408.0,29.83,0.10,30.00,9791342.0,9.017767,12.132494
412,TRPL4,23.53,22.50,35.00,20513277.0,23.51,0.10,50.00,5651512.0,0.085070,262.969715
263,MGLU3,22.88,0.01,25.00,19044065.0,24.79,0.01,25.00,12064443.0,-7.704720,57.852832
108,CRFB3,17.40,3.52,17.59,1656335.0,17.59,0.02,19.21,793122.0,-1.080159,108.837354
11,ALPA4,11.06,11.06,11.06,2200656.0,11.06,10.80,12.00,452011.0,0.000000,386.858948
25,ASAI3,9.06,0.01,15.00,4442560.0,5.41,0.01,15.00,10781117.0,67.467652,58.793138
120,CVCB3,8.26,0.01,40.00,7136020.0,2.54,0.01,15.00,7225418.0,225.196850,1.237271
238,LJQQ3,4.80,0.01,5.00,1818741.0,4.94,4.90,5.50,1582944.0,-2.834008,14.896105


In [126]:
filtered_df = filtered_df[filtered_df['dif taxa']!=0]

In [127]:
filtered_df.sort_values(['dif taxa','dif volume'])

,codigo,taxa media,taxa minima,taxa max,negocios,taxa media d-2,taxa minima d-2,taxa max d-2,negocios d-2,dif taxa,dif volume
5,AERI3,0.40,0.01,1.39,6098726.0,1.39,1.39,1.39,732625.0,-71.223022,732.448524
211,ITSA4,0.06,0.01,0.11,4881829.0,0.11,0.01,0.15,4635950.0,-45.454545,5.303746
34,B3SA3,0.05,0.01,1.09,10009357.0,0.09,0.01,0.20,17827751.0,-44.444444,43.855190
185,HAPV3,0.08,0.01,0.12,9281923.0,0.12,0.01,0.30,17533987.0,-33.333333,47.063249
298,PETR4,0.05,0.01,0.25,5724018.0,0.07,0.01,1.08,8904461.0,-28.571429,35.717412
148,EQTL3,0.08,0.01,1.11,1968399.0,0.11,0.01,1.12,2205670.0,-27.272727,10.757321
213,ITUB4,0.03,0.01,0.10,6139116.0,0.04,0.01,0.10,8005893.0,-25.000000,23.317536
143,EMBR3,0.03,0.01,0.04,1686602.0,0.04,0.02,1.00,1328936.0,-25.000000,26.913711
4,ABEV3,0.11,0.01,1.14,4728625.0,0.14,0.10,1.14,7289224.0,-21.428571,35.128554
113,CSNA3,0.08,0.01,0.25,1831868.0,0.10,0.02,0.20,4240393.0,-20.000000,56.799570


In [121]:
filtered_df.sort_values('dif volume')

,codigo,taxa media,taxa minima,taxa max,negocios,taxa media d-2,taxa minima d-2,taxa max d-2,negocios d-2,dif taxa,dif volume
25,ASAI3,9.06,0.01,15.00,4442560.0,5.41,0.01,15.00,10781117.0,67.467652,-58.793138
113,CSNA3,0.08,0.01,0.25,1831868.0,0.10,0.02,0.20,4240393.0,-20.000000,-56.799570
185,HAPV3,0.08,0.01,0.12,9281923.0,0.12,0.01,0.30,17533987.0,-33.333333,-47.063249
34,B3SA3,0.05,0.01,1.09,10009357.0,0.09,0.01,0.20,17827751.0,-44.444444,-43.855190
298,PETR4,0.05,0.01,0.25,5724018.0,0.07,0.01,1.08,8904461.0,-28.571429,-35.717412
4,ABEV3,0.11,0.01,1.14,4728625.0,0.14,0.10,1.14,7289224.0,-21.428571,-35.128554
99,COGN3,0.21,0.01,0.23,2177545.0,0.23,0.23,0.23,3251863.0,-8.695652,-33.037001
10,ALOS3,0.11,0.01,0.12,1650389.0,0.12,0.10,0.14,2252845.0,-8.333333,-26.742008
213,ITUB4,0.03,0.01,0.10,6139116.0,0.04,0.01,0.10,8005893.0,-25.000000,-23.317536
61,BOVA11,0.56,0.01,3.00,2903669.0,0.57,0.01,1.50,3539939.0,-1.754386,-17.974039


,codigo,taxa media,taxa minima,taxa max,negocios,taxa media d-2,taxa minima d-2,taxa max d-2,negocios d-2,dif taxa
120,CVCB3,8.26,0.01,40.00,7136020.0,2.54,0.01,15.00,7225418.0,225.196850
406,TOTS3,1.19,0.01,2.50,1761518.0,0.42,0.01,0.70,1635153.0,183.333333
25,ASAI3,9.06,0.01,15.00,4442560.0,5.41,0.01,15.00,10781117.0,67.467652
72,BRFS3,32.52,0.01,40.00,8603408.0,29.83,0.10,30.00,9791342.0,9.017767
286,OIBR3,2.12,2.10,2.50,1791458.0,2.10,2.10,2.10,904905.0,0.952381
412,TRPL4,23.53,22.50,35.00,20513277.0,23.51,0.10,50.00,5651512.0,0.085070
297,PETR3,0.15,0.01,0.15,2459552.0,0.15,0.15,0.20,2257777.0,0.000000
40,BBDC4,0.05,0.01,0.50,10304458.0,0.05,0.01,0.60,14747094.0,0.000000
173,GMAT3,0.05,0.01,0.05,2621147.0,0.05,0.05,0.05,650890.0,0.000000
299,PETZ3,3.91,0.01,3.91,2873716.0,3.91,3.91,3.91,7645990.0,0.000000
